In [1]:
import times_series_learning as tsl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from datetime import datetime
import time
from logreader import read_dir
from logparser import parse_all
from loganalyzer import MessageLogAnalyzer, describe, print_description, print_clusters
import class_time_series
import pandas as pd
import numpy as np


In [2]:
pwd

'/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/LogKiller'

In [3]:
LogPath = '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics'
p = class_time_series.GetServerData(LogPath)
ServerName = p.ServerList()

In [4]:
#Server present the 19/10/2017
ServerPresent1910 =ServerName.columns[ServerName.loc['2017-10-19'] !=0]
ServerPresent1910

Index(['10.60.64.53', 'airfrance.fr', 'centos.localdomain', 'g',
       'qviqbocpmg04', 'qvirbkarno10', 'qvirnascgo02', 'qvirnmonlx02',
       'qvisbcld0084', 'qvisbcld0146.novalocal', 'qvisbcldaa20',
       'qvisbcldaa22', 'qvisbcldaa35', 'qvisbcldaa36', 'qvisbcldaa38',
       'qvisbcldaa46', 'qvisbcldaa60', 'qvisbcldaa62', 'qvisbcldaa64',
       'qvisbcldaa66', 'qvisbcldaa68', 'qvisbcldaa70', 'qvisbcldaa72',
       'qvisbcldaa74', 'qvisbcldaa76', 'qvisbcldaa78', 'qvisbcloud12',
       'qvisbcloud22', 'qvisbcloud24', 'qvisbcloud28', 'qvisbcloud30',
       'qvisbcloud46', 'qvisbcloud48', 'qvisbcloud54', 'qvisbcloud58',
       'qvisbcloud64', 'qvisbcloud72', 'qvisbcloud88', 'qvisbcloud92',
       'qvisbcloud98', 'redhat.localdomain', 'tlssbcld0055',
       'tlssbcld0055.novalocal'],
      dtype='object')

In [5]:
ServerName.loc[:,ServerPresent1910]

,10.60.64.53,airfrance.fr,centos.localdomain,g,qviqbocpmg04,qvirbkarno10,qvirnascgo02,qvirnmonlx02,qvisbcld0084,qvisbcld0146.novalocal,...,qvisbcloud54,qvisbcloud58,qvisbcloud64,qvisbcloud72,qvisbcloud88,qvisbcloud92,qvisbcloud98,redhat.localdomain,tlssbcld0055,tlssbcld0055.novalocal
2017-04-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#best choice qvirbkarno
log_data_list = ServerName.loc[:,'qvirbkarno10'].values
log_data_list = [i for i in log_data_list if i !=0 ]
log_data_list

['/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics/qvirbkarno10.20170930.log',
 '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics/qvirbkarno10.20171001.log',
 '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics/qvirbkarno10.20171002.log',
 '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics/qvirbkarno10.20171003.log',
 '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics/qvirbkarno10.20171004.log',
 '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics/qvirbkarno10.20171006.log',
 '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics/qvirbkarno10.20171008.log',
 '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics/qvirbkarno10.20171010.log',
 '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/analytics/qvirbkarno10.20171011.log',
 '/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/log

In [15]:


analyzer = MessageLogAnalyzer()

# Import des logs
logs = read_dir('../logs/analytics/*qvirbkarno10*')

# Parsing des logs en df
df_log = parse_all(logs)

# Extraction de la partie message
X = df_log.loc[:,2:].values

# Count des fréquences verticales
X_count = analyzer.fit_count(X) # "batch"

# Obtention des labels de clustering
y = analyzer.fit_clusterize(X,X_count)

# Stockage des labels de clustering
df_log['label'] = y

df_log.to_csv('dataframe.csv')

../logs/analytics/qvirbkarno10.20171024.log
../logs/analytics/qvirbkarno10.20171004.log
../logs/analytics/qvirbkarno10.20171006.log
../logs/analytics/qvirbkarno10.20171021.log
../logs/analytics/qvirbkarno10.20171019.log
../logs/analytics/qvirbkarno10.20171008.log
../logs/analytics/qvirbkarno10.20171002.log
../logs/analytics/qvirbkarno10.20171020.log
../logs/analytics/qvirbkarno10.20170930.log
../logs/analytics/qvirbkarno10.20171026.log
../logs/analytics/qvirbkarno10.20171016.log
../logs/analytics/qvirbkarno10.20171011.log
../logs/analytics/qvirbkarno10.20171001.log
../logs/analytics/qvirbkarno10.20171003.log
../logs/analytics/qvirbkarno10.20171018.log
../logs/analytics/qvirbkarno10.20171010.log

[Sucess] 16 files loaded
[Success] Logs parsed within 2.09s
[Success] Vocabulary processed within 2.96s
[Success] 143049 Logs reduced into 70 Clusters within 38.11s


In [16]:
df_log.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-10-24 00:00:11,qvirbkarno10,sshd[18258]:,Did,not,receive,identification,string,from,387.687.387.951,None,...,None,None,None,None,None,None,None,None,None,14
2017-10-24 00:00:51,qvirbkarno10,xinetd[20976]:,START:,nrpe,pid,18697,from,387.547.159.271,None,None,...,None,None,None,None,None,None,None,None,None,0
2017-10-24 00:00:51,qvirbkarno10,xinetd[20976]:,EXIT:,nrpe,status,0,pid,18697,duration,0(sec),...,None,None,None,None,None,None,None,None,None,0
2017-10-24 00:01:01,qvirbkarno10,CROND[19477]:,(root),CMD,(run-parts /etc/cron.hourly),None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,8
2017-10-24 00:01:01,qvirbkarno10,run-parts(/etc/cron.hourly)[19477]:,starting,0anacron,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,10
